In [86]:
import pandas as pd
import chardet
import os
import re
from datetime import datetime

pd.set_option("display.max_columns", None)

Definir les fichiers

In [87]:
# 📌 Définir les dossiers
DOSSIER_VENDOR = "vendor_central_mois"
DOSSIER_AMVISOR = "amvisor_mois"
DOSSIER_DATA = "processed"

In [88]:
fichiers_amvisor = os.listdir(DOSSIER_AMVISOR)
fichiers_vendor_central = os.listdir(DOSSIER_VENDOR)

print(fichiers_amvisor)
print(fichiers_vendor_central)

['Export Hama (FR) 2025-01-31 EN.csv', '.DS_Store', 'Export Hama (FR) 2024-11-30 EN.csv', 'Export Hama (FR) 2024-12-31 EN.csv', 'Export Hama (FR) 2025-02-28 EU.csv', 'Export Hama (FR) 2024-10-31 EN.csv', 'Export Hama (FR) 2025-03-20 EU.csv']
['Ventes_ASIN_Fabrication_Venteaudétail_France_Mensuel_01-01-2025_31-01-2025.csv', 'Ventes_ASIN_Approvisionnement_Venteaudétail_France_Mensuel_01-12-2024_31-12-2024.csv', 'Ventes_ASIN_Approvisionnement_Venteaudétail_France_Mensuel_01-11-2024_30-11-2024.csv', 'Ventes_ASIN_Approvisionnement_Venteaudétail_France_Mensuel_01-02-2025_28-02-2025.csv', 'Ventes_ASIN_Fabrication_Venteaudétail_France_Mensuel_01-12-2024_31-12-2024.csv', 'Ventes_ASIN_Fabrication_Venteaudétail_France_Mensuel_01-11-2024_30-11-2024.csv', 'Ventes_ASIN_Fabrication_Venteaudétail_France_Mensuel_01-02-2025_28-02-2025.csv', 'Ventes_ASIN_Approvisionnement_Venteaudétail_France_Mensuel_01-01-2025_31-01-2025.csv']


Global func

In [89]:
# Fonction pour extraire la date (YYYY-MM) d'un fichier
def extraire_mois_annee(nom_fichier):
    # Format AMVisor (YYYY-MM)
    match_amvisor = re.search(r"(\d{4})-(\d{2})", nom_fichier)
    if match_amvisor:
        return match_amvisor.group(0)
    
    # Format Vendor Central (JJ-MM-YYYY_JJ-MM-YYYY)
    match_vendor = re.search(r"(\d{2})-(\d{2})-(\d{4})_\d{2}-\d{2}-\d{4}", nom_fichier)
    if match_vendor:
        return f"{match_vendor.group(3)}-{match_vendor.group(2)}"  

    return None

# Fonction pour récupérer la dernière date disponible entre les 2 dossiers
def get_last_available_date():
    fichiers = os.listdir(DOSSIER_VENDOR) + os.listdir(DOSSIER_AMVISOR)
    dates = {extraire_mois_annee(f) for f in fichiers if extraire_mois_annee(f)}
    
    if not dates:
        raise ValueError("❌ Aucune date valide trouvée dans les fichiers.")

    return max(dates)  # Retourne la dernière date (YYYY-MM)

# Fonction pour récupérer les fichiers d'un mois donné et vérifier leur présence
def get_files(annee_mois):
    fichiers_vendor = os.listdir(DOSSIER_VENDOR)
    fichiers_amvisor = os.listdir(DOSSIER_AMVISOR)

    # Filtrer les fichiers correspondant au mois donné
    fichiers_trouves = [
        os.path.join(DOSSIER_AMVISOR, f) for f in fichiers_amvisor if extraire_mois_annee(f) == annee_mois
    ] + [
        os.path.join(DOSSIER_VENDOR, f) for f in fichiers_vendor if extraire_mois_annee(f) == annee_mois
    ]

    # Vérification : on doit avoir exactement 3 fichiers (Export, Fabrication, Approvisionnement)
    if len(fichiers_trouves) != 3:
        print(f"❌ Fichiers manquants pour {annee_mois} : trouvés {len(fichiers_trouves)}/3.") # raise FileNotFoundError

    return fichiers_trouves


In [90]:
derniere_date = get_last_available_date()
fichiers_du_mois = get_files(derniere_date)

dates = ["2024-11", "2024-12", "2025-01", "2025-02"]
files = get_files("2025-02")
files

❌ Fichiers manquants pour 2025-03 : trouvés 1/3.


['amvisor_mois/Export Hama (FR) 2025-02-28 EU.csv',
 'vendor_central_mois/Ventes_ASIN_Approvisionnement_Venteaudétail_France_Mensuel_01-02-2025_28-02-2025.csv',
 'vendor_central_mois/Ventes_ASIN_Fabrication_Venteaudétail_France_Mensuel_01-02-2025_28-02-2025.csv']

In [91]:
def process_amvisor(path):

    # Load Amvisor
    df = pd.read_csv(path, sep=None, engine='python', encoding="ISO-8859-1", dtype={"EAN": str})
    df = df.dropna(subset=["ASIN"])

    # Get columns
    columns_main = ["ASIN", 'Item no.', "EAN", "Item"]
    columns_overall = ["Extras", "Size", "Description Item", "Cat. 1", "Cat. 2", "Cat. 3", "Cat. 4", "Visible", "Title Content", "Variations", "Brand store URL", "Images", "Videos", "AI summary", "Code", "Catalogue"]
    columns_ads = ["Ads Impressions CM", "Ads Clicks CM", "Ads CTR CM", "Ads Units 14d CM", "Ads Costs CM", "Ads RoAS CM", "Ads CVR CM"]
    columns_sales = ["Sell-out CM", "Sell-out PM", 'Total sell-out CM', 'Total sell-out PM', "Revenue CM", "Revenue PM", "Units CM", "Units PM", "SRP", "Margin", "Replacements CM", "Replacements PM"]
    columns_stocks = ['Stock', 'Stock value', 'Total stock', 'Total stock value']
    columns_state = ["Reviews", "Stars",'Coverage', 'Rank 1', 'Buy Box', 'Buy Box PM', 'Days not Buy Box', "Views CM", "Views PM", "CVR CM", "CVR PM"]
    all_columns = columns_main + columns_overall + columns_ads + columns_sales + columns_stocks + columns_state
    all_columns = [col for col in all_columns if col in df.columns]
    
    # Transform columns
    colonnes_a_convertir = columns_ads + columns_sales + columns_stocks + columns_state
    colonnes_a_convertir = [col for col in colonnes_a_convertir if col != "Buy Box"]
    df[colonnes_a_convertir] = df[colonnes_a_convertir].replace({"€": "", "%": "", "\u202f": "", ",": "."}, regex=True).astype(float)

    # Select data
    df = df[all_columns]

    return df

def process_vendor_central(path1, path2):

    # Load data
    df_fab = pd.read_csv(path1, skiprows=1)
    df_app = pd.read_csv(path2, skiprows=1)

    # Select columns
    columns = ["ASIN", "Nom du produit", "Marque", "COGS expédié", "COGS expédié – Période antérieure (%)", "COGS expédié – Même période l'année dernière (%)", "Unités expédiées", "Unités expédiées – Période antérieure (%)", "Unités expédiées – Même période l'année dernière (%)", "Retours client", "Retours du client – Période antérieure (%)", "Retours du client – Même période l'année dernière (%)"]
    df_fab = df_fab[columns]
    df_app = df_app[columns]

    # Merge data
    df = pd.concat([df_fab, df_app])

    # Convert data
    colonnes_a_convertir = [
        "COGS expédié", "COGS expédié – Période antérieure (%)", "COGS expédié – Même période l'année dernière (%)",
        "Unités expédiées", "Unités expédiées – Période antérieure (%)", "Unités expédiées – Même période l'année dernière (%)",
        "Retours client", "Retours du client – Période antérieure (%)", "Retours du client – Même période l'année dernière (%)"
    ]
    df[colonnes_a_convertir] = df[colonnes_a_convertir].replace({"€": "", "%": "", "\u202f": "", ",": "."}, regex=True).astype(float)

    # Remove duplicates based on COGS
    df = df.sort_values(by=["ASIN", "COGS expédié"], ascending=[True, False])
    df = df.drop_duplicates(subset="ASIN", keep="first")

    # Rename columns
    columns = ["ASIN", "Nom du produit", "Marque", "COGS", "COGS evol LM", "COGS evol SPLY", "Unités", "Unités evol LM", "Unités evol SPLY", "Retours", "Retours evol LM", "Retours evol SPLY"]
    df.columns = columns

    # Compute before evols
    df["COGS LM"] = (df["COGS"] / (1 + df["COGS evol LM"] / 100)).round(2)
    df["Unités LM"] = (df["Unités"] / (1 + df["Unités evol LM"] / 100)).round().fillna(0).astype(int)
    df["Retours LM"] = df["Retours"] / (1 + df["Retours evol LM"] / 100)

    return df

def process_amazon(df_amvisor, df_vendor_central):

    # Merge data
    df = pd.merge(df_vendor_central, df_amvisor, how="outer", left_on="ASIN", right_on="ASIN")
    print(f"amvisor : {len(df_amvisor)} | vendor_central : {len(df_vendor_central)} | merged : {len(df)}")
    
    return df


def processing(dates, DOSSIER_DATA = "processed"):
    all_data = []

    for date in dates:
        print(f"-- Processing {date} ...")
        files = get_files(date)
        amvisor = process_amvisor(files[0])
        vendor_central = process_vendor_central(files[1],files[2])
        data = process_amazon(amvisor, vendor_central)
        data.to_csv(f"{DOSSIER_DATA}/data_{date}.csv")

        data["extract_date"] = date
        all_data.append(data)

    final_df = pd.concat(all_data, ignore_index=True)
    final_df.to_csv(f"{DOSSIER_DATA}/all_data.csv")
    return final_df

In [92]:
files = get_files("2025-02")
amvisor = process_amvisor(files[0])
vendor_central = process_vendor_central(files[1],files[2])
data = process_amazon(amvisor, vendor_central)

print(f"{vendor_central['ASIN'].nunique()} unique ASIN and {vendor_central.shape[0]} rows")
print(f"{amvisor['ASIN'].nunique()} unique ASIN and {amvisor.shape[0]} rows")
print(f"{data['ASIN'].nunique()} unique ASIN and {data.shape[0]} rows")

amvisor : 2583 | vendor_central : 1252 | merged : 2834
1252 unique ASIN and 1252 rows
2583 unique ASIN and 2583 rows
2834 unique ASIN and 2834 rows


In [93]:
dates = ["2025-02", "2025-01", "2024-12","2024-11"]
data = processing(dates)

-- Processing 2025-02 ...
amvisor : 2583 | vendor_central : 1252 | merged : 2834
-- Processing 2025-01 ...
amvisor : 2225 | vendor_central : 1294 | merged : 2533
-- Processing 2024-12 ...
amvisor : 2234 | vendor_central : 1279 | merged : 2532
-- Processing 2024-11 ...
amvisor : 2251 | vendor_central : 1269 | merged : 2532
